In [1]:
import pandas as pd

def load_bio_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    
    sentences = []
    current_sentence = ""
    
    label={}
    
    temp_label_name = ""
    for line in lines:
        # print(line)
        line = line.replace('\n','')
        line = line.strip()
        # print(line)
        if not line:  # 空行表示句子结束
            if current_sentence:
                # 处理最后一个词
                if len(temp_label_name)>0:
                    label[temp_label_name] = temp_label_value
                    if temp_label_value=="":
                        print(current_sentence)
                    temp_label_name=""
                strLabel = ""
                for k in label:
                    strLabel = strLabel + str(k) + ":" + str(label[k])+";"
                sentences.append({'text':current_sentence,'label':str(label)})
                # sentences.append({'text':current_sentence,'label':label})
                current_sentence = ""
                label={}
            continue
        
        parts = line.split()
        current_sentence = current_sentence + parts[0]  # 字符
        if len(parts)>=2 and parts[1] == 'O' and len(temp_label_name)>0:
            label[temp_label_name] = temp_label_value
            temp_label_name=""
        elif len(parts)>=2 and parts[1][0] =='B':
            temp_label_name = parts[0]
            temp_label_value =parts[1][2:]
            
        elif len(parts)>=2 and parts[1][0] =='I':
            temp_label_name = temp_label_name + parts[0]
    # 处理最后一个句子
    if (current_sentence, label):
        sentences.append({'text':current_sentence,'label':label})
    
    return sentences

# 示例加载
train = load_bio_data('data/medical.train')
dev = load_bio_data('data/medical.dev') 
test = load_bio_data('data/medical.dev') 
train[0]

{'text': '现头昏口苦', 'label': "{'口苦': '临床表现'}"}

In [2]:
import pandas as pd
train_df = pd.DataFrame(train)
dev_df = pd.DataFrame(dev)
test_df = pd.DataFrame(test)

train_df.to_csv("data/train.csv",index=False)
dev_df.to_csv("data/dev.csv",index=False)
test_df.to_csv("data/test.csv",index=False)

# train = pd.read_csv("data/train.csv")

模型输出的数据处理，用于计算F1分数

In [6]:
import ast
from typing import Set, Tuple, List

def parse_entity_str(entity_str: str) -> Set[Tuple[str, str]]:
    """
    将字典格式的字符串解析为(实体文本, 实体类型)的二元组集合
    :param entity_str: 模型输出/真实标签的字典字符串，如"{'口苦': '临床表现'}"
    :return: 实体二元组集合，如{('口苦', '临床表现')}
    """
    try:
        # 解析字典字符串为Python字典（处理单引号字典）
        entity_dict = ast.literal_eval(entity_str)
        # 转换为字典
        entity_set = {}
        for entity_text, entity_type in entity_dict.items():
            # 过滤空实体/空类型（可选，根据你的数据情况调整）
            if entity_text.strip() and entity_type.strip():
                entity_set[entity_text.strip()] = entity_type.strip()
        return entity_set
    except (SyntaxError, ValueError, TypeError) as e:
        # 解析失败（如模型输出格式错误），返回空集合
        print(f"解析失败：{entity_str}，错误：{e}")
        return {}

In [7]:
dict = parse_entity_str( "{'功能性消化不良': '西医诊断'}")

In [ ]:
label = dict['功能性消化不良']

'西医诊断'

In [9]:
dataset={}

In [ ]:
if label not in dataset:
    dataset[label] = {"FN":0,"FP":0,"TP":0} 
# 精确率 = 找对的实体数（TP） / 模型总共说找到的实体数（TP+FP）
# 召回率 = 找对的实体数（TP） / 句子里真实的实体总数（TP+FN）
# TP:本来是，模型预测是
# FP本来是，模型说不是 - 漏检测
# FN本来不是，模型说是 - 误检测

# 筛选出相同key的
if v1 == v2:
    dataset[v1]["TP"]+=1
elif v1!=v2:
    dataset[v1]["FP"]+=1
    if v2 in dataset:
        dataset[v2]["FN"]+=1
# 处理不同的key
dataset[label[k1]]["FP"]+=1
if pred[k2] in dataset:
        dataset[v2]["FN"]+=1

